In [2]:
import numpy as np

from lamp import *
from lamp.utils.usps import *

In [9]:
datax = np.array([
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [1, 0, 0, 0]
])
datay = np.array([0, 1, 3]).reshape(-1, 1)
yhat = np.array([0, 1, 0]).reshape(-1, 1)

In [5]:
linear = Linear(8, 4)

In [6]:
print(linear._parameters)

[[-0.9943498   0.42694027  0.73562096  0.74386915]
 [ 0.97997391  0.93781426 -0.74071407  0.46017222]
 [-0.50133464  0.08294563  0.6792956   0.17137832]
 [ 0.66919269  0.05195024  0.56939877  0.21128158]
 [ 0.12486533 -0.39019738  0.31366972 -0.87480364]
 [-0.90999609  0.85201162 -0.88305539  0.25875809]
 [ 0.86865677  0.81514557 -0.28407609 -0.8611146 ]
 [-0.78974533 -0.93973058 -0.25972511 -0.75280341]]
